This file is to transform different data sets to match the input requirements of the Salesforce DataLoader.

Just importing packages

In [24]:
import pandas as pd
from os import listdir
from os.path import isfile, join

Get all the file names in the folder spreadsheets

In [25]:
onlyfiles = [f for f in listdir("spreadsheets/") if isfile(join("spreadsheets/", f))]
onlyfiles

['AB 2664 Campus Quarterly Update ALL (2).xlsx',
 'AB264 Jul17_Jan18.xlsx',
 'CITRIS Data 2018 for AB2664.xlsx',
 'contactextracts.csv',
 'Disco 2017 Fall UGBA Roster.csv',
 'IPIRA 2018 Startups.xlsx',
 'Lean Transfer 17 class list.xlsx',
 's1.csv',
 'SCET Fall 2016 Course Enrollment.xlsx',
 'SkyDeck Master List .xlsx',
 'success.csv']

Figuring out if the files are either xlsx files or csv files and parsing them according to their type.

In [26]:
allfiles = {}
for a in onlyfiles:
    if ".xlsx" in a:
        allfiles[a.split(' ', 1)[0]] = pd.ExcelFile("spreadsheets/" + a)
    if ".csv" in a:
        print(a)
        allfiles[a.split(' ', 1)[0]] = pd.read_csv("spreadsheets/" + a, encoding='ISO-8859-1')

contactextracts.csv
Disco 2017 Fall UGBA Roster.csv
s1.csv
success.csv


Create a dictionary where each sheet is process if we are dealing with .xlsx file.
The format is alldata['first word of the file name']['name of the individual sheets']
For csv files, it would be alldata['first word of the file name']['first word of the file name'] just repeated twice.

In [27]:
alldata = {}
for key, b in allfiles.items():
    if type(b) is pd.ExcelFile:
        alldata[key] = {}
        for a in b.sheet_names:
            alldata[key][a] = b.parse(a)
    if type(b) is pd.DataFrame:
        alldata[key] = {}
        alldata[key][key] = b
print(alldata.keys())

dict_keys(['AB', 'AB264', 'CITRIS', 'contactextracts.csv', 'Disco', 'IPIRA', 'Lean', 's1.csv', 'SCET', 'SkyDeck', 'success.csv'])


Example of looking at the file 'AB264 Jul17_Jan18.xlsx', tab 'SP 2018'

In [28]:
alldata['AB264']['SP 2018'].head()

,Term,Session,Class Nbr,Subject,Catalog Nbr,Section,Component,Descr,Name
0,2182,Regular,36315,INDENG,95,1,COL,NEWTON LECT SERIES,"Modi,Mahek R"
1,2182,Regular,36315,INDENG,95,1,COL,NEWTON LECT SERIES,"Buckingham,Henry Otis"
2,2182,Regular,36315,INDENG,95,1,COL,NEWTON LECT SERIES,"Vera,Alonso"
3,2182,Regular,36315,INDENG,95,1,COL,NEWTON LECT SERIES,"Cho,Alexander Saewon"
4,2182,Regular,36315,INDENG,95,1,COL,NEWTON LECT SERIES,"Rietdyk,Arthur Julian"


In [29]:
campaigns = pd.DataFrame()
compiler = []
for key, value in alldata['AB264'].items():
    temp = []
    for a in value.groupby(['Class Nbr', 'Subject', 'Catalog Nbr', "Section", "Descr"]).sum().index.tolist():
        if (key == 'SP 2018'):
            temp.append((a + ('Spring 2018',)))
        else:
            temp.append((a + (key,)))
    compiler.extend(temp)
campaigns = pd.DataFrame(compiler, columns = ["Class Nbr", "Subject", "Catalog Nbr", "Section", "Descr", "Semester"])
campaigns["type"] = "I&E Course"
campaigns["Department"] = "SCET - Sutardja Center for Entrepreneurship & Technology"
campaigns["Industry"] = "SCET - Sutardja Center for Entrepreneurship & Technology"
campaigns["Class Name"] = campaigns["Subject"] + " " + campaigns["Catalog Nbr"].map(str) + " "+ campaigns["Semester"]

In [30]:
campaignlist = list(campaigns)
campaignlist.remove("Class Nbr")
campaignlist.remove("Section")
a = campaigns.groupby(campaignlist)["Class Nbr"].apply(list)
a = a.reset_index()

In [31]:
a.to_csv("ab264UniqueClasses3", index = False)

In [32]:
alldata["Lean"]['sheet1'].head()

,Term,Session,Class Nbr,Combined Sections ID,Subject,Catalog Nbr,Section,Component,Descr,Name,...,Enrollment Status,Last Update Date,Last Update Time,Related 1 Class Nbr,Related 1 Section,Related 1 Component,Related 2 Class Nbr,Related 2 Section,Related 2 Component,Ctlg Nbr Sort
0,2178,Regular,46372,1301,INDENG,190E,1,SEM,ADVANCE TOPICS IEOR,"Chen,Guanzhang",...,Enrolled,2017-08-04,2017-09-20 12:15:12,0.0,NaN,NaN,0.0,NaN,NaN,190E
1,2178,Regular,46372,1301,INDENG,190E,1,SEM,ADVANCE TOPICS IEOR,"Doshi,Vir",...,Enrolled,2017-09-01,2017-09-20 10:51:13,0.0,NaN,NaN,0.0,NaN,NaN,190E
2,2178,Regular,46372,1301,INDENG,190E,1,SEM,ADVANCE TOPICS IEOR,"Grossberg,Rachel Sonia",...,Enrolled,2017-07-28,2017-09-20 14:30:11,0.0,NaN,NaN,0.0,NaN,NaN,190E
3,2178,Regular,46372,1301,INDENG,190E,1,SEM,ADVANCE TOPICS IEOR,"Hou,Jason Frank",...,Enrolled,2017-07-27,2017-09-20 17:53:32,0.0,NaN,NaN,0.0,NaN,NaN,190E
4,2178,Regular,46372,1301,INDENG,190E,1,SEM,ADVANCE TOPICS IEOR,"Quraishi,Annam Khalid",...,Enrolled,2017-08-14,2017-09-20 09:26:06,0.0,NaN,NaN,0.0,NaN,NaN,190E


In [33]:
alldata['AB264']['SP 2018'].head()

,Term,Session,Class Nbr,Subject,Catalog Nbr,Section,Component,Descr,Name
0,2182,Regular,36315,INDENG,95,1,COL,NEWTON LECT SERIES,"Modi,Mahek R"
1,2182,Regular,36315,INDENG,95,1,COL,NEWTON LECT SERIES,"Buckingham,Henry Otis"
2,2182,Regular,36315,INDENG,95,1,COL,NEWTON LECT SERIES,"Vera,Alonso"
3,2182,Regular,36315,INDENG,95,1,COL,NEWTON LECT SERIES,"Cho,Alexander Saewon"
4,2182,Regular,36315,INDENG,95,1,COL,NEWTON LECT SERIES,"Rietdyk,Arthur Julian"


In [34]:
a['Season'] = a['Semester'].str.split(' ').str.get(0)
a['Year'] = a['Semester'].str.split(' ').str.get(1)
a.head()

,Subject,Catalog Nbr,Descr,Semester,type,Department,Industry,Class Name,Class Nbr,Season,Year
0,INDENG,185,CHALLENGE LAB,Summer 2017,I&E Course,SCET - Sutardja Center for Entrepreneurship & ...,SCET - Sutardja Center for Entrepreneurship & ...,INDENG 185 Summer 2017,[14336],Summer,2017
1,INDENG,192,BME BOOTCAMP,Summer 2017,I&E Course,SCET - Sutardja Center for Entrepreneurship & ...,SCET - Sutardja Center for Entrepreneurship & ...,INDENG 192 Summer 2017,[14339],Summer,2017
2,INDENG,135,APPLIED DATA SCI,Fall 2017,I&E Course,SCET - Sutardja Center for Entrepreneurship & ...,SCET - Sutardja Center for Entrepreneurship & ...,INDENG 135 Fall 2017,[47035],Fall,2017
3,INDENG,135,APPLIED DATA SCI,Spring 2018,I&E Course,SCET - Sutardja Center for Entrepreneurship & ...,SCET - Sutardja Center for Entrepreneurship & ...,INDENG 135 Spring 2018,[41878],Spring,2018
4,INDENG,171,TECH FIRM LEADRSHIP,Fall 2017,I&E Course,SCET - Sutardja Center for Entrepreneurship & ...,SCET - Sutardja Center for Entrepreneurship & ...,INDENG 171 Fall 2017,[39374],Fall,2017


In [35]:
ab264data = pd.DataFrame()
for key, value in alldata['AB264'].items():
    temp = []
    for b in value['Session']:
        if (key == 'SP 2018'):
            temp.append('Spring 2018')
        else:
            temp.append(key)
    value['SemesterY'] = temp
    ab264data = pd.concat([ab264data, value])
ab264data["type"] = "I&E Course"
ab264data["Department"] = "SCET - Sutardja Center for Entrepreneurship & Technology"
ab264data["Industry"] = "SCET - Sutardja Center for Entrepreneurship & Technology"
ab264data["Class Name"] = ab264data["Subject"] + " " + ab264data["Catalog Nbr"].map(str) + " "+ ab264data["SemesterY"]
ab264data['Course_Type'] = ab264data['Component'].replace(['LEC', 'SEM', 'COL'], ['Lecture', 'Seminar', 'Colloquium'])
ab264data['FirstName'] = ab264data['Name'].str.split(',').str.get(1)
ab264data['LastName'] = ab264data['Name'].str.split(',').str.get(0)
ab264data['FullName'] = ab264data['FirstName'] + " " + ab264data['LastName']
ab264data['Semester'] = ab264data['SemesterY'].str.split(' ').str.get(0)
ab264data['Year'] = ab264data['SemesterY'].str.split(' ').str.get(1)
ab264data["Campaign_Name"] = "SCET " + ab264data['Class Name']
ab264data["Contact Tag"] = 'I&E Center Student'
ab264data["Contact Type"] = "Student"
ab264data["Center ID"] = "a0mG00000083sT6"
ab264data["Record Type"] = "012G0000001BHHg"
ab264data['AccountID'] = "001G000001ZiZc6"
output = alldata['s1.csv']['s1.csv']
output = output.groupby(["CLASS NAME", "ID"]).sum().reset_index()[["CLASS NAME", "ID"]]
ab264data = ab264data.merge(output, left_on='Class Name', right_on='CLASS NAME', how='inner')

In [36]:
ab264temp = ab264data
ab264data = ab264data.drop(columns=["Term", "Session", "Class Nbr", "Subject", "Catalog Nbr", "Section", "Component", "Descr", "Name", "SemesterY", "Department", "Industry"])

In [37]:
tempdf = alldata['contactextracts.csv']['contactextracts.csv']
tempdf.columns

Index(['LASTNAME', 'FIRSTNAME'], dtype='object')

In [38]:
a.head()

,Subject,Catalog Nbr,Descr,Semester,type,Department,Industry,Class Name,Class Nbr,Season,Year
0,INDENG,185,CHALLENGE LAB,Summer 2017,I&E Course,SCET - Sutardja Center for Entrepreneurship & ...,SCET - Sutardja Center for Entrepreneurship & ...,INDENG 185 Summer 2017,[14336],Summer,2017
1,INDENG,192,BME BOOTCAMP,Summer 2017,I&E Course,SCET - Sutardja Center for Entrepreneurship & ...,SCET - Sutardja Center for Entrepreneurship & ...,INDENG 192 Summer 2017,[14339],Summer,2017
2,INDENG,135,APPLIED DATA SCI,Fall 2017,I&E Course,SCET - Sutardja Center for Entrepreneurship & ...,SCET - Sutardja Center for Entrepreneurship & ...,INDENG 135 Fall 2017,[47035],Fall,2017
3,INDENG,135,APPLIED DATA SCI,Spring 2018,I&E Course,SCET - Sutardja Center for Entrepreneurship & ...,SCET - Sutardja Center for Entrepreneurship & ...,INDENG 135 Spring 2018,[41878],Spring,2018
4,INDENG,171,TECH FIRM LEADRSHIP,Fall 2017,I&E Course,SCET - Sutardja Center for Entrepreneurship & ...,SCET - Sutardja Center for Entrepreneurship & ...,INDENG 171 Fall 2017,[39374],Fall,2017


In [39]:
success = alldata['success.csv']['success.csv']
success["contactID"] = success['ID']
success = success.drop(columns=['ID', 'ID.1'])
success['campaignId'] = ab264data['ID']

In [40]:
campaigns = ab264temp.drop_duplicates(subset = ["Class Name"])

In [41]:
campaigns.to_csv("campaignidss.csv", index = False)

In [42]:
success.to_csv("campaignmembers.csv", index = False)